# E-Commerce Project(Pandas):

In [1]:
!pip install numpy
!pip install pandas
!pip install pyodbc

In [40]:
import numpy as np

In [41]:
from warnings import filterwarnings
import pyodbc
import pandas as pd

filterwarnings('ignore')

# Define Connection
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-IDSEH902\\SQLEXPRESS;'
    'DATABASE=Hospital;'
    'Trusted_Connection=yes;'
)

# Query the Table
query = "SELECT * FROM Ecommerce"
df = pd.read_sql_query(query, conn)

# Display the Data
print(df.head())

   OrderID  CustomerID   OrderDate  ProductID ProductCategory ProductName  \
0   100001        4174  2024-05-28        370          Beauty      Item E   
1   100002        4507  2024-11-21        169  Home & Kitchen      Item A   
2   100003        1860  2022-09-29        284            Toys      Item E   
3   100004        2294  2024-04-15        327            Toys      Item C   
4   100005        2130  2023-01-03        168           Books      Item D   

   Quantity  PricePerUnit  Discount PaymentMethod CustomerLocation  \
0         2    237.429993      0.00           UPI          Central   
1         3    415.809998      0.05   Credit Card          Central   
2         7    475.220001      0.22           UPI            North   
3         8    226.440002      0.26   Credit Card             West   
4         7    209.880005      0.10   Net Banking             West   

   DeliveryTimeDays DeliveryStatus  SellerID  Rating   TotalPrice  
0                14      Delivered       282    

In [41]:
from warnings import filterwarnings
import pyodbc
import pandas as pd

filterwarnings('ignore')

# Define Connection
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-IDSEH902\\SQLEXPRESS;'
    'DATABASE=Hospital;'
    'Trusted_Connection=yes;'
)

# Query the Table
query = "SELECT * FROM Ecommerce"
df = pd.read_sql_query(query, conn)

# Display the Data
print(df.head())

   OrderID  CustomerID   OrderDate  ProductID ProductCategory ProductName  \
0   100001        4174  2024-05-28        370          Beauty      Item E   
1   100002        4507  2024-11-21        169  Home & Kitchen      Item A   
2   100003        1860  2022-09-29        284            Toys      Item E   
3   100004        2294  2024-04-15        327            Toys      Item C   
4   100005        2130  2023-01-03        168           Books      Item D   

   Quantity  PricePerUnit  Discount PaymentMethod CustomerLocation  \
0         2    237.429993      0.00           UPI          Central   
1         3    415.809998      0.05   Credit Card          Central   
2         7    475.220001      0.22           UPI            North   
3         8    226.440002      0.26   Credit Card             West   
4         7    209.880005      0.10   Net Banking             West   

   DeliveryTimeDays DeliveryStatus  SellerID  Rating   TotalPrice  
0                14      Delivered       282    

**Analyze Seasonal Order Behaviour Across Years:**
- Question – 1 :
- As an E-Commerce analytics expert, you are tasked with understanding customer Purchasing behaviour over time, Your goal is to determine in which quarters (Q1 –Q4) customers places the most orders and how that trend has evolved from 2022 to 2024. This insights will help stakeholders to plan marketing and inventory strategies around high-demand periods. Construct a feature to identify the quarter from the order date and aggregate the total number of orders per quarter, per year.

In [42]:
#Change the Data Type of OrderDate from obj - Date:
df['OrderDate'] = pd.to_datetime(df['OrderDate'])

In [43]:
df.dtypes

OrderID                      int64
CustomerID                   int64
OrderDate           datetime64[ns]
ProductID                    int64
ProductCategory             object
ProductName                 object
Quantity                     int64
PricePerUnit               float64
Discount                   float64
PaymentMethod               object
CustomerLocation            object
DeliveryTimeDays             int64
DeliveryStatus              object
SellerID                     int64
Rating                     float64
TotalPrice                 float64
dtype: object

In [44]:
#Feature Engineering:

df['Year'] = df['OrderDate'].dt.year
df['Quarter'] = df['OrderDate'].dt.to_period('Q')

#See How many Orders has been placed by Quarter & year:
quarter = df.groupby(['Year','Quarter']).size().to_frame()

#Sorting the Data for Better Readability
quarterly_orders = quarter.sort_values(by=['Year', 'Quarter'])
quarterly_orders

0
Year Quarter      
2022 2022Q1   3262
     2022Q2   3369
     2022Q3   3487
     2022Q4   3420
2023 2023Q1   3267
     2023Q2   3326
     2023Q3   3362
     2023Q4   3238
2024 2024Q1   3377
     2024Q2   3283
     2024Q3   3290
     2024Q4   3319

**Business Insights**

1.This Analysis helps us to discover Peak periods.
2.From the analysis we can conclude 2024 has the Peak period from overall Data.
3.The reason behind the peak it might indicate festival shopping behaviour.
4.This can help with planning discount campaigns and optimizing the inventory.


**Calculate Seller-Wise Average Product Rating for Performance BenchMarking:**
- Question – 2: 
- The Quality of Service and Product Satisfaction is essential for long-term customer 
retention in an eCommerce business. As a Data Analyst, you need to provide sellerlevel performance insights to the marketplace operations team. Calculate the 
average customer rating received by each seller, along with the total number of 
products sold by that seller. This will help identify high-performing sellers and those 
who may need support or training interventions.

In [45]:
# Seller countPerformance:-

df.groupby('SellerID').agg(

    AverageRating = ('Rating', 'mean'),
    TotalOrders = ('OrderID', 'count')
).sort_values(by = ['AverageRating'],ascending=False)

,AverageRating,TotalOrders
SellerID,,
211,3.160167,359
269,3.135733,375
226,3.114987,387
258,3.114721,394
288,3.097594,374
...,...,...
256,2.910000,410
291,2.901527,393
293,2.888281,384


**Business Insights**
1. This analysis helps to categorize seller into Top-tiers.
2. Top-Rated seller can be incentivized with premium visibility.
3. Low-Rated seller will be flagged with feedbacks to improve for the scalability.

**Identify Product Categories with the Highest Return and Cancellation Rates**
- Question – 3 :
- Returns and cancellations significantly impact logistics costs and customer 
satisfaction. To improve the supply chain and reduce losses, identify which product 
categories have the highest percentage of returned or cancelled orders. This metric 
should be normalized based on the total number of orders in each category to 
account for volume differences. Your output should include the total number of 
orders, return/cancel counts, and return/cancel rate per category.

1. Total Numbers of Orders.
2. Return/Cancel counts.
3. Return/Cancel rate per category.

In [46]:
df['IsReturnOrCancelled'] = df['DeliveryStatus'].isin(['Returned',
                                                      'Cancelled'])

In [51]:
df.columns

Index(['OrderID', 'CustomerID', 'OrderDate', 'ProductID', 'ProductCategory',
       'ProductName', 'Quantity', 'PricePerUnit', 'Discount', 'PaymentMethod',
       'CustomerLocation', 'DeliveryTimeDays', 'DeliveryStatus', 'SellerID',
       'Rating', 'TotalPrice', 'Year', 'Quarter', 'IsReturnOrCancelled'],
      dtype='object')

In [53]:
#Group By Product Analysis:
category_return_analysis = df.groupby('ProductCategory').agg(

    TotalOrders = ('OrderID', 'count'),
    ReturnCancelCount = ('IsReturnOrCancelled', 'sum')
)

# Calculate Cancel / Return Rate
category_return_analysis['ReturnCancelRate (%)']=(category_return_analysis['ReturnCancelCount']/category_return_analysis['TotalOrders'])*100

#Sort the Items
category_return_analysis = category_return_analysis.sort_values(
    by = 'ReturnCancelRate(%)', ascending = False),round(2)

category_return_analysis

KeyError: 'ReturnCancelRate(%)'

**Business Insights**
1. Product Category with High Return / Cancel Rate may indicate issue with the product mismatch or defects in the product.
2. This information is critical for category managers to investigate suppliers to have enhance product billings / Quality check before delivering the product.

In [25]:
df.columns

Index(['OrderID', 'CustomerID', 'OrderDate', 'ProductID', 'ProductCategory',
       'ProductName', 'Quantity', 'PricePerUnit', 'Discount', 'PaymentMethod',
       'CustomerLocation', 'DeliveryTimeDays', 'DeliveryStatus', 'SellerID',
       'Rating', 'TotalPrice', 'Year', 'Quarter'],
      dtype='object')

**Analyze Top 10 High-Revenue Products with Adjusted Revenue Considering Discounts**

Question – 4:
- Revenue insights are vital for identifying product success. As a revenue analyst, your
task is to determine the top 10 products that generate the most revenue after
discounts are applied. The metric should use a calculated TotalPrice that adjusts for
discounts, not just the MRP. This will help product and pricing teams understand
which products are driving real monetary value.


In [5]:
df.columns

Index(['OrderID', 'CustomerID', 'OrderDate', 'ProductID', 'ProductCategory',
       'ProductName', 'Quantity', 'PricePerUnit', 'Discount', 'PaymentMethod',
       'CustomerLocation', 'DeliveryTimeDays', 'DeliveryStatus', 'SellerID',
       'Rating', 'TotalPrice'],
      dtype='object')

In [4]:
# From Exponential Values to Real Float Values
pd.options.display.float_format = '{:.2f}'.format

In [5]:
# We have to calculate total adjusted revenue:
revenue = df.groupby('ProductName').agg(

    TotalRevenue = ('TotalPrice', 'sum'),
    TotalQuantitySold = ('Quantity', 'sum')
)

top_products = revenue.sort_values(by ='TotalRevenue',
                                  ascending = False)

top_products

,TotalRevenue,TotalQuantitySold
ProductName,,
Item D,8623426.91,40202
Item A,8619518.74,40311
Item E,8543835.73,39452
Item B,8402288.49,39459
Item C,8364208.87,39616


In [6]:
df['TotalPrice']

0        474.86
1       1185.06
2       2594.70
3       1340.52
4       1322.24
          ...  
39995   2177.99
39996   2095.05
39997   2090.57
39998     18.10
39999    649.77
Name: TotalPrice, Length: 40000, dtype: float64

In [ ]:
price of the product and total price of the product

**Business Insights**:
1. An Accurate picture of product profitability through this analysis.
2. This can be useful for cross-selling() or promotional placement.

**Evaluate Regional Order Distribution and Total Revenue for Strategic Expansion**
Question – 5:
- Understanding geographic trends is essential for regional marketing and logistics
planning. As part of a regional strategy evaluation, calculate the total number of
orders and the total revenue generated from each region (CustomerLocation). This
will support decisions regarding warehouse expansion, delivery optimization, and
targeted marketing campaigns.

In [23]:
region_analysis = df.groupby('CustomerLocation').agg(

    TotalOrders = ('OrderID', 'count'),
    TotalRevenue = ('TotalPrice', 'sum')
)

region_analysis = region_analysis.sort_values(by = 'TotalRevenue',
                                              ascending = False)
region_analysis

,TotalOrders,TotalRevenue
CustomerLocation,,
Central,8048,8583031.53
West,8029,8573505.36
East,8046,8547505.44
South,7910,8427529.52
North,7967,8421706.89


**Business Insights**
1. Region with High Order Volume and revenue (Eg.Central, West) can be prioritized for faster delivery hubs or regional Discounts.
2. While we implement in the required Location , Customer engagement raises rapidly.
3. Ofcourse, seller onbording also promoted if incase we consider this criteria.

**Determine Average Delivery Time by Product Category to Identify Supply Chain**
Bottlenecks
Question – 6:
- In eCommerce logistics, average delivery time is a crucial KPI that reflects supply
chain efficiency. Management wants to understand which product categories are
taking longer to deliver than others. Your goal is to compute the average delivery
time in days per product category. This will help logistics teams identify bottlenecks
and improve delivery SLAs.

In [7]:
category_delivery_time = df.groupby('ProductCategory').agg(

    AverageDeliveryTime = ('DeliveryTimeDays', 'mean'),
    TotalOrders = ('OrderID', 'count')
)

category_delivery_time = category_delivery_time.sort_values(by = 'AverageDeliveryTime',
                                                            ascending = False)

category_delivery_time

,AverageDeliveryTime,TotalOrders
ProductCategory,,
Home & Kitchen,7.56,6634
Electronics,7.52,6572
Books,7.49,6765
Beauty,7.48,6653
Clothing,7.48,6751
Toys,7.41,6625


**Business Insights**
1. Longer Delivery times in certain (Home & Kitchen, Electronics)map point to fulfilment issues.
2. It depends on Supplier dependencies or geographical constraints.

**Identify Customer Segments Based on Purchase Volume for Loyalty Program Targeting**

Question – 7:
- To design an effective loyalty or rewards program, it's essential to classify customers
based on their total order volume. Segment the customers into three tiers—Low,
Medium, and High—based on the number of orders they've placed.

- This will help the marketing team focus their campaigns on loyal, high-value customers while also re-
engaging low-activity ones.

In [14]:
# Step 1 - Total Orders
customer_orders = df.groupby('CustomerID').agg(

    TotalOrders = ('OrderID', 'count')
)

# Step 2 - high, Medium, Low:
def segment_customers(order_count):
    if order_count >= 50:
        return 'High'
    elif (order_count >= 20 and order_count < 50):
        return 'Medium'
    else:
        return 'Low'

# Step 3 - 
customer_orders['CustomerSegment'] = [segment_customers(x) for x in customer_orders['TotalOrders']]

segement_distribution = customer_orders['CustomerSegment'].value_counts().to_frame()
segement_distribution

,count
CustomerSegment,
Low,3981
Medium,19


**Business Insights**
1. There is no Avail High-Tier Customers from the Analysis.
2. So we can get focused 
3. Which can make the customers to engage towards personalized fundamentals.

**Calculate Net Effective Discount Percentage Offered Across Categories**

Question – 8 :
- Discounts are a key part of eCommerce pricing strategies. However, stakeholders
want a realistic picture of the average net effective discount given in each product
category, not just the highest advertised rate. Your task is to calculate the average
discount percentage offered across each category and determine which categories
are operating at higher markdowns. This analysis aids pricing teams in managing
margin leakage.

In [16]:
category_discount = df.groupby('ProductCategory').agg(

    Average_Discount = ('Discount', 'mean')
)
category_discount['Average Discount (%)'] = category_discount['Average_Discount']*100

category_discount = category_discount.sort_values(by = 'Average Discount (%)',
                                                  ascending = False)
category_discount

,Average_Discount,Average Discount (%)
ProductCategory,,
Clothing,0.15,15.20
Toys,0.15,15.09
Beauty,0.15,15.03
Home & Kitchen,0.15,15.02
Books,0.15,14.99
Electronics,0.15,14.89


**Identify Top 5 Cities (Regions) with Highest Average Order Value (AOV)**
- Question – 9 :
- Marketing and regional operations teams need clarity on customer purchasing power 
across regions. One of the best indicators of this is Average Order Value (AOV)—the 
average revenue generated per order. Your goal is to compute AOV for each customer 
region and return the top 5 regions. This will help teams target affluent markets with 
premium products and personalized campaigns.

In [61]:
# Group by CustomerLocation and calculate AOV
aov_by_city = (
    df.groupby("CustomerLocation")["TotalPrice"]
    .mean()
    .reset_index(name="AverageOrderValue")
)

#Sort by AOV in descending order and get top 5
top_5_cities = aov_by_city.sort_values(by="AverageOrderValue", ascending=False).head(5)

# Display the result
print(top_5_cities)

  CustomerLocation  AverageOrderValue
4             West        1067.817332
0          Central        1066.480061
3            South        1065.427247
1             East        1062.329783
2            North        1057.073791


**Business Insights**

1. Engineer a feature to identify repeat customers (those with more than one order).
2. Analyze how much total revenue.

**Engineer a Feature to Identify Repeat Customers and Analyze Their Contribution to Revenue**
- Question – 10 :
- Repeat customers are the backbone of a successful eCommerce business. To help the 
customer success team evaluate loyalty, create a new feature that identifies repeat 
customers—those who have placed more than one order. Then, compute how much 
of the total revenue comes from these repeat customers compared to first-time 
buyers. This insight will validate the ROI of retention strategies and justify loyalty 
program investments.


In [60]:
#Count orders per customer
customer_order_counts = df.groupby("CustomerID")["OrderID"].nunique().reset_index()
customer_order_counts.rename(columns={"OrderID": "OrderCount"}, inplace=True)

#Label each customer as 'Repeat' or 'First-Time'
customer_order_counts["CustomerType"] = customer_order_counts["OrderCount"].apply(
    lambda x: "Repeat" if x > 1 else "First-Time"
)

#Merge this back to the original DataFrame
df = df.merge(customer_order_counts[["CustomerID", "CustomerType"]], on="CustomerID", how="left")

#Analyze revenue by customer type
revenue_by_type = (
    df.groupby("CustomerType")["TotalPrice"]
    .agg(Total_Revenue="sum", Order_Count="count")
    .reset_index()
)

#Calculate % contribution
total_revenue = revenue_by_type["Total_Revenue"].sum()
revenue_by_type["Revenue_Contribution_%"] = (
    100 * revenue_by_type["Total_Revenue"] / total_revenue
).round(2)

# Display the result
print(revenue_by_type)

  CustomerType  Total_Revenue  Order_Count  Revenue_Contribution_%
0       Repeat   4.255328e+07        40000                   100.0


**Businees Insights**
1. Visualize this revenue split between repeat and first-time buyers.

**Identify Top 5 Sellers by Revenue and Analyze Their Average Product Ratings**
- Question – 11 :
- Revenue is critical, but coupling it with customer satisfaction gives a complete picture 
of seller performance. Your goal is to identify the top 5 sellers in terms of total 
revenue, and then evaluate their average product ratings. This will help the 
marketplace team reward high-value sellers who maintain quality, and investigate 
those who may be sacrificing customer experience for volume.

In [62]:
#Aggregate total revenue and average rating per seller
seller_summary = (
    df.groupby("SellerID")
    .agg(
        Total_Revenue=("TotalPrice", "sum"),
        Avg_Rating=("Rating", "mean")
    )
    .reset_index()
)

#Get Top 5 sellers by revenue
top_5_sellers = seller_summary.sort_values(by="Total_Revenue", ascending=False).head(5)

# Round for readability
top_5_sellers["Avg_Rating"] = top_5_sellers["Avg_Rating"].round(2)

print(top_5_sellers)


    SellerID  Total_Revenue  Avg_Rating
25       225  518947.115975        2.94
35       235  500242.896962        2.89
89       289  494221.885525        3.02
9        209  480991.295777        3.09
29       229  479393.252439        3.09


**Business Insights**
1. Comparing revenue and rating for these top sellers.

**Analyze Impact of Payment Methods on Average Order Value and Return Rates**
- Question – 12 :
- Finance and operations teams want to assess how different payment methods affect 
purchasing behavior and return patterns. Your task is to calculate two key metrics for 
each payment method: Average Order Value (AOV) and Return/Cancel Rate. This will 
inform strategic decisions such as incentivizing certain payment methods or 
identifying friction points in others.

In [63]:
#Flag returns or cancellations
df["Is_Returned"] = df["DeliveryStatus"].isin(["Returned", "Cancelled"]).astype(int)

#Group by PaymentMethod and calculate metrics
payment_method_analysis = (
    df.groupby("PaymentMethod")
    .agg(
        Avg_Order_Value=("TotalPrice", "mean"),
        Total_Orders=("OrderID", "count"),
        Returned_Orders=("Is_Returned", "sum")
    )
    .reset_index()
)

#Calculate Return/Cancel Rate (%)
payment_method_analysis["Return_Cancel_Rate_%"] = (
    100 * payment_method_analysis["Returned_Orders"] / payment_method_analysis["Total_Orders"]
).round(2)

#Round AOV for readability
payment_method_analysis["Avg_Order_Value"] = payment_method_analysis["Avg_Order_Value"].round(2)

# Optional: Drop intermediate columns if not needed
payment_method_analysis = payment_method_analysis[[
    "PaymentMethod", "Avg_Order_Value", "Return_Cancel_Rate_%"
]]

# Display result
print(payment_method_analysis)

      PaymentMethod  Avg_Order_Value  Return_Cancel_Rate_%
0  Cash on Delivery          1066.27                 50.14
1       Credit Card          1060.86                 49.60
2        Debit Card          1062.29                 50.14
3       Net Banking          1062.12                 49.44
4               UPI          1067.58                 50.92


**Businees Insights**
1. Compare AOV vs. return rate!

**Engineer a Flag for High-Value Orders and Analyze Their Distribution Across Categories.**
- Question – 13 :
- Product and marketing teams are interested in identifying "high-value" transactions 
to offer premium customer experiences (e.g., free shipping, early access). You need 
to engineer a new feature that flags high-value orders—defined as those with a total 
price greater than the 90th percentile of all orders. Then, analyze how these orders 
are distributed across product categories.

In [75]:
#Calculate the 90th percentile of TotalPrice
p90 = df["TotalPrice"].quantile(0.90)

#Create the high-value order flag
df["IsHighValueOrder"] = (df["TotalPrice"] > p90).astype(int)

#Analyze distribution of high-value orders across categories
high_value_distribution = (
    df.groupby("ProductCategory")["IsHighValueOrder"]
    .agg(High_Value_Orders="sum", Total_Orders="count")
    .reset_index()
)

#Add % of high-value orders in each category
high_value_distribution["High_Value_Order_%"] = (
    100 * high_value_distribution["High_Value_Orders"] / high_value_distribution["Total_Orders"]
).round(2)

# Display result
print(high_value_distribution.sort_values(by="High_Value_Order_%", ascending=False))


  ProductCategory  High_Value_Orders  Total_Orders  High_Value_Order_%
3     Electronics                692          6572               10.53
5            Toys                695          6625               10.49
2        Clothing                672          6751                9.95
1           Books                664          6765                9.82
4  Home & Kitchen                647          6634                9.75
0          Beauty                630          6653                9.47


**Business Insights**
1.  visualize which categories drive the most high-value orders!

**Calculate Average Rating by Product Category and Identify Underperforming Segments**
- Question – 14 :
- Customer reviews are critical indicators of product satisfaction. As part of a quality 
assurance initiative, you are asked to calculate the average rating for each product 
category. The goal is to flag categories where the average rating falls below 3.5, 
indicating potential customer dissatisfaction. This will help product teams prioritize 
improvements or reconsider inventory from specific suppliers.

In [64]:
#Calculate average rating by category
category_rating = (
    df.groupby("ProductCategory")["Rating"]
    .mean()
    .reset_index(name="Avg_Rating")
)

#Flag underperforming categories
category_rating["Is_Underperforming"] = category_rating["Avg_Rating"] < 3.5

#Optional - Round the average rating for readability
category_rating["Avg_Rating"] = category_rating["Avg_Rating"].round(2)

#Filter only underperforming if needed
underperforming_categories = category_rating[category_rating["Is_Underperforming"]]

print(category_rating.sort_values(by="Avg_Rating"))


  ProductCategory  Avg_Rating  Is_Underperforming
3     Electronics        2.99                True
2        Clothing        3.00                True
4  Home & Kitchen        3.00                True
5            Toys        3.00                True
0          Beauty        3.01                True
1           Books        3.01                True


**Business Insights**
1. Visualize the rating distribution.
2. Highlight top-rated categories as well!

**Engineer Product Profitability Metric and Identify Most and Least Profitable Categories.**
- Question – 15 :
- While revenue is important, profitability is what ultimately matters to the finance 
team. For better decision-making, you are tasked with engineering a new feature: 
Estimated Profit per order. Assume a flat product cost = 70% of the PricePerUnit 
(excluding discount). Using this, calculate estimated profit for each order and then 
aggregate the total profit by product category. Identify the top 3 most profitable and 
least profitable categories.

In [68]:
#Estimate product cost and profit
df["Estimated_Cost"] = 0.70 * df["PricePerUnit"] * df["Quantity"]
df["Estimated_Profit"] = df["TotalPrice"] - df["Estimated_Cost"]

#Aggregate total profit by category
profit_by_category = (
    df.groupby("ProductCategory")["Estimated_Profit"]
    .sum()
    .reset_index(name="Total_Profit")
    .sort_values(by="Total_Profit", ascending=False)
)

#Identify top 3 and bottom 3 categories
top_3 = profit_by_category.head(3)
bottom_3 = profit_by_category.tail(3)

result = pd.concat([top_3, bottom_3], axis=0).reset_index(drop=True)
result["Total_Profit"] = result["Total_Profit"].round(2)

print(result)

  ProductCategory  Total_Profit
0           Books    1276220.46
1     Electronics    1270457.37
2            Toys    1244562.25
3  Home & Kitchen    1235731.41
4        Clothing    1235219.66
5          Beauty    1231951.86


**Business Insights**
1. The most and least profitable categorie.
2. Factor in return losses.

**Engineer Order Completion Flag and Analyze Seller Fulfillment Efficiency**
- Question – 16 :
- To assess seller reliability, operations teams want a metric that reflects the fulfillment 
efficiency of each seller. Begin by engineering a new feature: an Order Completion 
Flag which is True only if the delivery status is “Delivered”. Then, calculate the Order 
Completion Rate (%) for each seller by dividing completed orders by total orders. Sort 
the sellers from best to worst based on this fulfillment metric.

In [67]:
#Create Order Completion Flag
df["Order_Completed"] = df["DeliveryStatus"].apply(lambda x: True if x == "Delivered" else False)

#Group by SellerID to calculate completion rate
seller_fulfillment = (
    df.groupby("SellerID")
    .agg(
        Total_Orders=("OrderID", "count"),
        Completed_Orders=("Order_Completed", "sum")
    )
    .reset_index()
)

#Calculate completion rate %
seller_fulfillment["Completion_Rate_%"] = (
    100 * seller_fulfillment["Completed_Orders"] / seller_fulfillment["Total_Orders"]
).round(2)

#Sort from best to worst
seller_fulfillment = seller_fulfillment.sort_values(by="Completion_Rate_%", ascending=False)

print(seller_fulfillment.head(10))  # top 10 reliable sellers

    SellerID  Total_Orders  Completed_Orders  Completion_Rate_%
78       278           394               120              30.46
98       298           390               114              29.23
10       210           404               118              29.21
70       270           384               111              28.91
85       285           417               119              28.54
90       290           374               106              28.34
33       233           387               109              28.17
64       264           398               112              28.14
13       213           410               115              28.05
89       289           435               122              28.05


**Business Insights**
1. Engineer a new column Order_Completed — True only if DeliveryStatus == 'Delivered'.
2. Group by SellerID to calculate Order Completion Rate.
3. Sort sellers from best to worst based on fulfillment.

**Determine Category-Wise Revenue Growth Year Over Year.**
- Question – 17 :
- Understanding growth trends across product categories helps in strategic investment 
decisions. The executive team is interested in knowing which product categories are 
showing positive or negative revenue growth year over year. Calculate the total 
revenue per category for each year, and then derive a feature for Year-over-Year (YoY) 
Growth (%) to highlight accelerating or declining segments.

In [66]:
# Extract Year from OrderDate
df["Year"] = df["OrderDate"].dt.year

#Aggregate total revenue per ProductCategory per Year
category_yearly_revenue = (
    df.groupby(["ProductCategory", "Year"])["TotalPrice"]
    .sum()
    .reset_index(name="Total_Revenue")
    .sort_values(by=["ProductCategory", "Year"])
)

#Calculate YoY Growth % for each category
category_yearly_revenue["YoY_Growth_%"] = (
    category_yearly_revenue
    .groupby("ProductCategory")["Total_Revenue"]
    .pct_change() * 100
).round(2)

#
print(category_yearly_revenue)

   ProductCategory  Year  Total_Revenue  YoY_Growth_%
0           Beauty  2022   2.329629e+06           NaN
1           Beauty  2023   2.320030e+06         -0.41
2           Beauty  2024   2.348137e+06          1.21
3            Books  2022   2.397491e+06           NaN
4            Books  2023   2.429950e+06          1.35
5            Books  2024   2.415745e+06         -0.58
6         Clothing  2022   2.438307e+06           NaN
7         Clothing  2023   2.351372e+06         -3.57
8         Clothing  2024   2.321888e+06         -1.25
9      Electronics  2022   2.372034e+06           NaN
10     Electronics  2023   2.372961e+06          0.04
11     Electronics  2024   2.385747e+06          0.54
12  Home & Kitchen  2022   2.450911e+06           NaN
13  Home & Kitchen  2023   2.346206e+06         -4.27
14  Home & Kitchen  2024   2.195802e+06         -6.41
15            Toys  2022   2.386865e+06           NaN
16            Toys  2023   2.355332e+06         -1.32
17            Toys  2024   2

**Business Insights**

1. Plot this as a line chart showing revenue growth trends.
2. Filter only negative or positive growth categories.

**Use List Comprehension to Classify Products Based on Price Ranges for Marketing Segmentation.**
- Question – 18 :
- To drive targeted pricing campaigns, the marketing team wants products categorized 
based on their unit price. Create a new column that classifies each product into Low, 
Mid, or High price segments using list comprehension. Then, analyze how much total 
revenue each segment contributes to the business. This helps in targeting different 
customer segments effectively.
Low: PricePerUnit < ₹100
Mid: ₹100 ≤ PricePerUnit ≤ ₹300
High: PricePerUnit > ₹300*/

In [65]:
#Create 'PriceSegment' using list comprehension
df["PriceSegment"] = [
    "Low" if price < 100 
    else "Mid" if 100 <= price <= 300 
    else "High"
    for price in df["PricePerUnit"]
]

#Analyze revenue contribution by each segment
segment_revenue = (
    df.groupby("PriceSegment")["TotalPrice"]
    .sum()
    .reset_index(name="Total_Revenue")
)

#Add % contribution for context
total_revenue = segment_revenue["Total_Revenue"].sum()
segment_revenue["Revenue_Contribution_%"] = (
    100 * segment_revenue["Total_Revenue"] / total_revenue
).round(2)

print(segment_revenue.sort_values(by="Revenue_Contribution_%", ascending=False))

  PriceSegment  Total_Revenue  Revenue_Contribution_%
0         High   2.730895e+07                   64.18
2          Mid   1.353792e+07                   31.81
1          Low   1.706408e+06                    4.01


**Business Insights**
1. Visualize this with a pie chart or bar plot for reporting!

**Identify the Return Rate of High-Value Orders Compared to Low-Value Orders.**
Question – 19 :
- Customer behavior can vary significantly based on the value of a transaction. The 
operations and fraud detection teams want to compare the return/cancel rate of 
high-value orders (orders above the 75th percentile) with that of low-value orders 
(orders below the 25th percentile). This can help uncover patterns such as abuse of 
returns or product dissatisfaction in expensive or cheap items.


In [57]:
#Calculate percentiles

q25 = df["TotalPrice"].quantile(0.25)
q75 = df["TotalPrice"].quantile(0.75)

#Classify order value bands
df["OrderValueBand"] = [
    "Low" if price <= q25 
    else "High" if price >= q75 
    else "Mid"
    for price in df["TotalPrice"]
]

#Flag return/cancel orders
df["Is_Returned"] = df["DeliveryStatus"].isin(["Returned", "Cancelled"]).astype(int)

#Analyze return rate by order value band
return_rate = (
    df[df["OrderValueBand"].isin(["Low", "High"])]
    .groupby("OrderValueBand")
    .agg(
        Total_Orders=("OrderID", "count"),
        Returned_Orders=("Is_Returned", "sum")
    )
    .reset_index()
)

#Compute return/cancel rate (%)
return_rate["Return_Rate_%"] = (
    100 * return_rate["Returned_Orders"] / return_rate["Total_Orders"]
).round(2)

# Display result
print(return_rate.sort_values(by="OrderValueBand"))

  OrderValueBand  Total_Orders  Returned_Orders  Return_Rate_%
0           High         10000             5036          50.36
1            Low         10000             5022          50.22


**Business Insight**
1. Higher return rate in Low-value orders may indicate fraud or dissatisfaction.
2. Lower return rate in High-value orders may show more serious buyers.

**Engineer Delivery Performance Bands and Analyze Their Impact on Customer Ratings.**
- Question – 20 :
- To improve last-mile delivery and customer satisfaction, the logistics team wants to 
understand how delivery time impacts customer ratings. Your task is to engineer a 
new feature called DeliveryPerformanceBand that classifies delivery speed as Fast 
(≤3 days), Moderate (4–7 days), or Slow (>7 days). Then analyze the average 
customer rating within each delivery band.

In [59]:
#Create DeliveryPerformanceBand using list comprehension
df["DeliveryPerformanceBand"] = [
    "Fast" if days <= 3 
    else "Moderate" if days <= 7 
    else "Slow"
    for days in df["DeliveryTimeDays"]
]

#Calculate average rating for each delivery band
delivery_rating_analysis = (
    df.groupby("DeliveryPerformanceBand")["Rating"]
    .mean()
    .reset_index(name="Avg_Customer_Rating")
    .sort_values(by="Avg_Customer_Rating", ascending=False)
)

# Round for clarity
delivery_rating_analysis["Avg_Customer_Rating"] = delivery_rating_analysis["Avg_Customer_Rating"].round(2)

# Display result
print(delivery_rating_analysis)

  DeliveryPerformanceBand  Avg_Customer_Rating
1                Moderate                 3.01
2                    Slow                 3.00
0                    Fast                 2.99


**Business Insights**
1. Higher ratings for fast delivery indicate better customer satisfaction.
2. Poor ratings for slow deliveries highlight areas for logistics improvement.

**Engineer a Feature to Flag Peak Season Orders and Compare Revenue Contribution.**
- Question – 21 :
- The sales team suspects that a large chunk of revenue is concentrated during the 
peak seasons like festive quarters (Q4) or year-end sales. Your job is to engineer a 
new feature called IsPeakSeason based on the order quarter, where Q4 is considered 
peak. Then calculate the total revenue and number of orders from peak vs. non-peak 
periods to validate the hypothesis.

In [74]:
#Extract Quarter from OrderDate
df["Quarter"] = df["OrderDate"].dt.quarter

#Create IsPeakSeason flag (Q4 is peak)
df["IsPeakSeason"] = df["Quarter"] == 4

#Group by Peak/Non-Peak and compute revenue and order count
peak_season_analysis = (
    df.groupby("IsPeakSeason")
    .agg(
        Total_Revenue=("TotalPrice", "sum"),
        Number_of_Orders=("OrderID", "count")
    )
    .reset_index()
)

#Rename True/False for clarity
peak_season_analysis["Season"] = peak_season_analysis["IsPeakSeason"].map({True: "Peak (Q4)", False: "Non-Peak"})

peak_season_analysis = peak_season_analysis[["Season", "Total_Revenue", "Number_of_Orders"]]

print(peak_season_analysis)


      Season  Total_Revenue  Number_of_Orders
0   Non-Peak   3.192116e+07             30023
1  Peak (Q4)   1.063212e+07              9977


**Business insights**
1. If Q4 revenue > Non-Peak, it confirms the sales team’s suspicion.
2. This can justify campaigns, discounts, or logistics planning during festive periods.

**Perform Multi-Metric Performance Analysis of Each Product Category**
- Question – 22 :
- The executive dashboard needs a composite view of each product category's 
business performance. You are required to compile a table showing the following for 
each category:
Total Revenue
Total Quantity Sold
Average Rating
Return/Cancel Rate
- This multi-metric view will help the leadership team identify not just top-selling 
categories, but also the ones that balance revenue, satisfaction, and operational 
efficiency

In [73]:
# Create a return/cancel flag
df["Is_Returned"] = df["DeliveryStatus"].isin(["Returned", "Cancelled"]).astype(int)

#Group by ProductCategory and calculate metrics
category_performance = (
    df.groupby("ProductCategory")
    .agg(
        Total_Revenue=("TotalPrice", "sum"),
        Total_Quantity_Sold=("Quantity", "sum"),
        Average_Rating=("Rating", "mean"),
        Total_Orders=("OrderID", "count"),
        Returned_Orders=("Is_Returned", "sum")
    )
    .reset_index()
)

#Calculate Return/Cancel Rate (%)
category_performance["Return_Cancel_Rate_%"] = (
    100 * category_performance["Returned_Orders"] / category_performance["Total_Orders"]
).round(2)


category_performance["Average_Rating"] = category_performance["Average_Rating"].round(2)
category_performance = category_performance.drop(columns=["Total_Orders", "Returned_Orders"])

print(category_performance.sort_values(by="Total_Revenue", ascending=False))

  ProductCategory  Total_Revenue  Total_Quantity_Sold  Average_Rating  \
1           Books   7.243186e+06                33712            3.01   
3     Electronics   7.130742e+06                32886            2.99   
2        Clothing   7.111567e+06                33529            3.00   
5            Toys   7.077066e+06                32783            3.00   
0          Beauty   6.997797e+06                33041            3.01   
4  Home & Kitchen   6.992920e+06                33089            3.00   

   Return_Cancel_Rate_%  
1                 49.67  
3                 50.67  
2                 50.07  
5                 50.91  
0                 49.23  
4                 49.76  


**Business Insights**
1. Add visualizations (e.g., heatmap, radar chart)
2. Filter for top-performing or underperforming categories based on custom rules.

**Segment Sellers by Revenue Tiers and Analyze Their Average Completion Rate**
- Question – 23 :
- To design tiered incentive programs for marketplace sellers, the business team needs to 
segment sellers into Low, Mid, and High Revenue groups. Then, calculate the average order 
completion rate within each revenue tier. This analysis will reveal whether top sellers are 
also the most reliable, or if mid/low-tier sellers outperform them in fulfillment quality.


In [ ]:
# Create order completion flag
df["Order_Completed"] = df["DeliveryStatus"].eq("Delivered").astype(int)

# Calculate total revenue and completion rate per seller
seller_stats = (
    df.groupby("SellerID")
    .agg(
        Total_Revenue=("TotalPrice", "sum"),
        Total_Orders=("OrderID", "count"),
        Completed_Orders=("Order_Completed", "sum")
    )
    .reset_index()
)

# Calculate completion rate (%)
seller_stats["Completion_Rate_%"] = (
    100 * seller_stats["Completed_Orders"] / seller_stats["Total_Orders"]
).round(2)

# Segment sellers into revenue tiers using quantiles
q1 = seller_stats["Total_Revenue"].quantile(0.33)
q2 = seller_stats["Total_Revenue"].quantile(0.66)

def segment_revenue_tier(revenue):
    if revenue <= q1:
        return "Low"
    elif revenue <= q2:
        return "Mid"
    else:
        return "High"

seller_stats["Revenue_Tier"] = seller_stats["Total_Revenue"].apply(segment_revenue_tier)

# Calculate average completion rate per revenue tier
tier_analysis = (
    seller_stats.groupby("Revenue_Tier")["Completion_Rate_%"]
    .mean()
    .reset_index(name="Avg_Completion_Rate_%")
    .sort_values(by="Avg_Completion_Rate_%", ascending=False)
)


print(tier_analysis)

**Business Insights**
1. Helps identify whether high-revenue sellers are also the most reliable.
2. You can design tier-wise incentives based on fulfillment behavior.

**Engineer a Customer Frequency Band and Analyze Their Contribution to Revenue**
- Question – 24 :
- Customer frequency is a core driver of lifetime value. You are asked to engineer a 
new feature called CustomerFrequencyBand, categorizing customers based on the 
number of orders they’ve placed:
Rare (1 order)
Occasional (2–4 orders)
Frequent (5+ orders)
- Then, analyze how much revenue each customer band contributes. This insight will 
help the CRM team design personalized loyalty campaigns and better manage 
customer retention efforts.


In [72]:
# Count orders per customer
customer_order_counts = (
    df.groupby("CustomerID")["OrderID"]
    .count()
    .reset_index(name="Total_Orders")
)

# Categorize frequency bands
def get_frequency_band(n):
    if n == 1:
        return "Rare"
    elif 2 <= n <= 4:
        return "Occasional"
    else:
        return "Frequent"

customer_order_counts["CustomerFrequencyBand"] = customer_order_counts["Total_Orders"].apply(get_frequency_band)

#  Merge band info back to main df
df = df.merge(customer_order_counts[["CustomerID", "CustomerFrequencyBand"]], on="CustomerID", how="left")

# Analyze revenue contribution by band
frequency_revenue = (
    df.groupby("CustomerFrequencyBand")["TotalPrice"]
    .sum()
    .reset_index(name="Total_Revenue")
)

# Add percentage contribution
total_revenue = frequency_revenue["Total_Revenue"].sum()
frequency_revenue["Revenue_Contribution_%"] = (
    100 * frequency_revenue["Total_Revenue"] / total_revenue
).round(2)

print(frequency_revenue.sort_values(by="Revenue_Contribution_%", ascending=False))

  CustomerFrequencyBand  Total_Revenue  Revenue_Contribution_%
0              Frequent   4.210631e+07                   98.95
1            Occasional   4.469646e+05                    1.05


**Business insights**
1. Frequent customers are the most valuable — deserving loyalty perks.
2. Rare buyers may need reactivation campaigns.

**Evaluate Seller-Level Discounting Behavior and Its Correlation with Revenue**
- Question – 25 :
- Discounting is often used as a lever to boost sales—but excessive discounting may 
eat into profits. The finance team wants to assess whether sellers offering higher 
average discounts are generating proportionately higher revenue. Calculate average 
discount percentage and total revenue for each seller, and rank them to identify 
outliers—sellers with high discounts but low revenue (or vice versa).


In [71]:
# Group by SellerID to get average discount and total revenue
seller_discount_analysis = (
    df.groupby("SellerID")
    .agg(
        Avg_Discount_Percentage=("Discount", "mean"),
        Total_Revenue=("TotalPrice", "sum")
    )
    .reset_index()
)

# Round for readability
seller_discount_analysis["Avg_Discount_Percentage"] = seller_discount_analysis["Avg_Discount_Percentage"].round(2)
seller_discount_analysis["Total_Revenue"] = seller_discount_analysis["Total_Revenue"].round(2)

# Sort by discount or revenue (optional)
# To find high-discount low-revenue sellers
seller_discount_analysis = seller_discount_analysis.sort_values(by=["Avg_Discount_Percentage", "Total_Revenue"], ascending=[False, True])

print(seller_discount_analysis)

    SellerID  Avg_Discount_Percentage  Total_Revenue
94       294                     0.16      374658.91
81       281                     0.16      392197.71
95       295                     0.16      396169.67
52       252                     0.16      400681.41
77       277                     0.16      405410.66
..       ...                      ...            ...
53       253                     0.14      435962.71
75       275                     0.14      437785.65
71       271                     0.14      441022.46
65       265                     0.14      451423.08
79       279                     0.14      469304.87

[100 rows x 3 columns]


**Business Insights**
1. Sellers in top-left (high discount, low revenue) may need review
2. Sellers with low discount, high revenue may have strong brand power